# Yoon Kim

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

import torchtext

from tensorboardX import SummaryWriter
from tqdm import tqdm as tqdm

np.random.seed(42)
CUDA = torch.cuda.is_available()

CUDA

True

In [3]:
# alphabet from the paper
# https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf
ALPHABET = ['\n'] + [s for s in """ abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&* ̃‘+-=<>()[]{}"""]

# Data preparation

In [ ]:
Чтобы использовать CNN на слова, нужно фи

# Model

In [ ]:
class YoonKimModel:
    def __init__(self, dropout):
        super(YoonKimModel, self).__init__()
        self.dropout = dropout
        self.embedding_dim = embedding_dim
        self.hidden_dim_1 = hidden_dim_1
        self.hidden_dim_2 = hidden_dim_2
        
        self.embedding = nn.Linear(len(ALPHABET), embedding_dim)
        self.chars_cnn = nn.
        self.words_rnn = nn.GRU(hidden_dim_1, hidden_dim_2, dropout=dropout)
        self.projector = nn.Linear(hidden_dim_2, 2)
        
    def forward(self, x):
        